### 커서 이동시 결과 출력

In [1]:
import cv2
import numpy as np

# 커서 아래의 픽셀의 HSV 값을 가져오기 위한 콜백 함수
def get_hsv_values(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        # 커서 아래 픽셀의 HSV 값을 가져옵니다.
        hsv_values = hsv_image[y, x]
        # 값이 지정된 범위 내에 있는지 확인합니다.
        is_within_range = (lower_green <= hsv_values) & (hsv_values <= upper_green)
        result = 1 if np.all(is_within_range) else 0
        # 결과를 이미지에 표시합니다.
        result_image = image.copy()
        result_image[green_mask != 0] = [0, 255, 0]  # 감지된 영역에 녹색 색상 적용
        cv2.putText(result_image, str(result), (image.shape[1] - 30, image.shape[0] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        # 이미지를 표시합니다.
        cv2.imshow('Detected_Green_Regions', result_image)

# 이미지 파일을 읽어옵니다.
image = cv2.imread('test3.jpg')

# 이미지를 HSV로 변환합니다.
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 녹색 식물 영역을 분리합니다.
lower_green = np.array([30, 30, 30])  # 하한값을 조정
upper_green = np.array([90, 255, 255])  # 상한값을 조정
green_mask = cv2.inRange(hsv_image, lower_green, upper_green)

# 녹색 영역에서 녹색 픽셀 수를 측정합니다.
green_pixels = np.count_nonzero(green_mask)

# 전체 픽셀 수를 계산합니다.
total_pixels = image.shape[0] * image.shape[1]

# 녹색 비율을 계산합니다.
green_ratio = green_pixels / total_pixels

# 녹색 픽셀 수를 출력합니다.
print("이미지 전체 픽셀 수: ", total_pixels)
print("식물 영역의 녹색 픽셀 수:", green_pixels)

# 녹색 비율을 출력합니다.
print("식물 영역의 녹색 비율: {:.2%}".format(green_ratio))

# 창과 콜백을 설정합니다.
cv2.namedWindow('Detected_Green_Regions')
cv2.setMouseCallback('Detected_Green_Regions', get_hsv_values)

# 초기 결과 이미지를 표시합니다.
result_image = image.copy()
result_image[green_mask != 0] = [0, 255, 0]  # 감지된 영역에 녹색 색상 적용
cv2.imshow('Detected_Green_Regions', result_image)

cv2.waitKey(0)
cv2.destroyAllWindows()


이미지 전체 픽셀 수:  252280
식물 영역의 녹색 픽셀 수: 5498
식물 영역의 녹색 비율: 2.18%


### 마우스 클릭시 원 생성, 드래그로 원 크기 조정, 원이 만들어진 이후 마우스 클릭시 클릭한 곳으로 원 이동

In [8]:
import cv2
import numpy as np

# 전역 변수 설정
center = (-1, -1)
drawing = False
radius = 0

# 마우스 이벤트 처리 함수
def on_mouse(event, x, y, flags, param):
    global center, drawing, radius

    if event == cv2.EVENT_LBUTTONDOWN:
        # 마우스 클릭 시작 지점 설정
        center = (x, y)
        drawing = True

    elif event == cv2.EVENT_LBUTTONUP:
        # 마우스 클릭 종료
        drawing = False

    elif event == cv2.EVENT_MOUSEMOVE:
        # 마우스 이동 중, 현재 그리고 있는 원의 크기 동적으로 조절
        if drawing:
            radius = int(np.sqrt((x - center[0])**2 + (y - center[1])**2))

# 이미지 파일을 읽어옵니다.
image = cv2.imread('test3.jpg')

# 이미지를 HSV로 변환합니다.
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 녹색 식물 영역을 분리합니다.
lower_green = np.array([30, 30, 30])  # 하한값을 조정
upper_green = np.array([90, 255, 255])  # 상한값을 조정
green_mask = cv2.inRange(hsv_image, lower_green, upper_green)

# 녹색 영역에서 녹색 픽셀 수를 측정합니다.
green_pixels = np.count_nonzero(green_mask)

# 전체 픽셀 수를 계산합니다.
total_pixels = image.shape[0] * image.shape[1]

# 녹색 비율을 계산합니다.
green_ratio = green_pixels / total_pixels
print("식물 영역의 녹색 비율: {:.2%}".format(green_ratio))

# 시각화를 위해 녹색 마스크를 원본 이미지에 오버레이합니다.
result_image = image.copy()
result_image[green_mask != 0] = [0, 255, 0]  # Green color for the detected regions

# 마우스 이벤트 콜백 함수 등록
cv2.namedWindow('Detected_Green_Regions')
cv2.setMouseCallback('Detected_Green_Regions', on_mouse)

while True:
    # 마우스 이벤트를 처리하며 계속해서 표시
    temp_image = result_image.copy()
    cv2.circle(temp_image, center, radius, (0, 0, 0), 2)
    cv2.imshow('Detected_Green_Regions', temp_image)

    key = cv2.waitKey(20) & 0xFF
    if key == 27:  # ESC 키를 누르면 종료
        break

cv2.destroyAllWindows()


식물 영역의 녹색 비율: 2.18%


#### 한 번 두개의 이벤트를 합쳐보는 곳

In [9]:
import cv2
import numpy as np
center = (-1, -1)
drawing = False
radius = 0

# 커서 아래의 픽셀의 HSV 값을 가져오기 위한 콜백 함수
def get_hsv_values(event, x, y, flags, param):
    global center, drawing, radius

    if event == cv2.EVENT_LBUTTONDOWN:
        # 마우스 클릭 시작 지점 설정
        center = (x, y)
        drawing = True

    elif event == cv2.EVENT_LBUTTONUP:
        # 마우스 클릭 종료
        drawing = False

    elif event == cv2.EVENT_MOUSEMOVE:
        hsv_values = hsv_image[y, x]
        # 값이 지정된 범위 내에 있는지 확인합니다.
        is_within_range = (lower_green <= hsv_values) & (hsv_values <= upper_green)
        result = 1 if np.all(is_within_range) else 0
        # 결과를 이미지에 표시합니다.
        result_image = image.copy()
        result_image[green_mask != 0] = [0, 255, 0]  # 감지된 영역에 녹색 색상 적용
        cv2.putText(result_image, str(result), (image.shape[1] - 30, image.shape[0] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        # 이미지를 표시합니다.
        cv2.imshow('Detected_Green_Regions', result_image)
        # 마우스 이동 중, 현재 그리고 있는 원의 크기 동적으로 조절
        if drawing:
            radius = int(np.sqrt((x - center[0])**2 + (y - center[1])**2))

# 이미지 파일을 읽어옵니다.
image = cv2.imread('test3.jpg')

# 이미지를 HSV로 변환합니다.
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 녹색 식물 영역을 분리합니다.
lower_green = np.array([30, 30, 30])  # 하한값을 조정
upper_green = np.array([90, 255, 255])  # 상한값을 조정
green_mask = cv2.inRange(hsv_image, lower_green, upper_green)

# 녹색 영역에서 녹색 픽셀 수를 측정합니다.
green_pixels = np.count_nonzero(green_mask)

# 전체 픽셀 수를 계산합니다.
total_pixels = image.shape[0] * image.shape[1]

# 녹색 비율을 계산합니다.
green_ratio = green_pixels / total_pixels

# 녹색 픽셀 수를 출력합니다.
print("이미지 전체 픽셀 수: ", total_pixels)
print("식물 영역의 녹색 픽셀 수:", green_pixels)

# 녹색 비율을 출력합니다.
print("식물 영역의 녹색 비율: {:.2%}".format(green_ratio))

# 창과 콜백을 설정합니다.
cv2.namedWindow('Detected_Green_Regions')
cv2.setMouseCallback('Detected_Green_Regions', get_hsv_values)

# 초기 결과 이미지를 표시합니다.
result_image = image.copy()
result_image[green_mask != 0] = [0, 255, 0]  # 감지된 영역에 녹색 색상 적용
while True:
    # 마우스 이벤트를 처리하며 계속해서 표시
    temp_image = result_image.copy()
    cv2.circle(temp_image, center, radius, (0, 0, 0), 2)
    cv2.imshow('Detected_Green_Regions', temp_image)

    key = cv2.waitKey(20) & 0xFF
    if key == 27:  # ESC 키를 누르면 종료
        break
cv2.destroyAllWindows()


이미지 전체 픽셀 수:  252280
식물 영역의 녹색 픽셀 수: 5498
식물 영역의 녹색 비율: 2.18%


In [78]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
%matplotlib qt5

x = np.linspace(0, 3*np.pi, 500)
plt.plot(x, np.sin(x**2))
plt.title('A simple chirp')
plt.show()

ImportError: Cannot load backend 'TkAgg' which requires the 'tk' interactive framework, as 'qt' is currently running